### urllib

In [2]:
import urllib

In [3]:
urllib??

urllib.request

urllib.error

urllib.parse

urllib.robotparser

urllib.response

In [4]:
from urllib import robotparser

robot = robotparser.RobotFileParser()
robot.set_url("https://www.google.com/robots.txt")
robot.read()
robot.can_fetch("Bot", "/search/about")

False

In [5]:
robot = robotparser.RobotFileParser()
robot.set_url("https://news.naver.com/robots.txt")
robot.read()
robot.can_fetch("Yeti", "/main/imagemontage")

True

In [6]:
robot = robotparser.RobotFileParser()
robot.set_url("http://gsen.hanyang.ac.kr/robots.txt")
robot.read()
robot.can_fetch("*", "/admin")

True

In [7]:
from urllib import request

In [12]:
resp = request.urlopen('https://www.google.com')

In [13]:
resp.geturl()

'https://www.google.com'

In [14]:
resp.reason

'OK'

In [15]:
resp.getcode()

200

In [17]:
resp.getheaders()

[('Date', 'Thu, 11 Jul 2019 02:10:13 GMT'),
 ('Expires', '-1'),
 ('Cache-Control', 'private, max-age=0'),
 ('Content-Type', 'text/html; charset=ISO-8859-1'),
 ('P3P', 'CP="This is not a P3P policy! See g.co/p3phelp for more info."'),
 ('Server', 'gws'),
 ('X-XSS-Protection', '0'),
 ('X-Frame-Options', 'SAMEORIGIN'),
 ('Set-Cookie',
  '1P_JAR=2019-07-11-02; expires=Sat, 10-Aug-2019 02:10:13 GMT; path=/; domain=.google.com'),
 ('Set-Cookie',
  'NID=187=Yw2_M_mhK_xcamR5PyWDhMVnuF5lt3Zw8nw7sEwnVdII3rCYo_VkEE1ZYHsgtkNGOe7alRpzBVOvAr5YSn9l7Il5PUfANsj9TDkFWTKDijjVatKhLHrvrJHLFaWj8dEu1xG7-Jd4V4CAESB6GoJjYztcCd4EC2DKI5HT-wrGQsY; expires=Fri, 10-Jan-2020 02:10:13 GMT; path=/; domain=.google.com; HttpOnly'),
 ('Alt-Svc', 'quic=":443"; ma=2592000; v="46,43,39"'),
 ('Accept-Ranges', 'none'),
 ('Vary', 'Accept-Encoding'),
 ('Connection', 'close')]

In [18]:
resp.status

200

In [19]:
resp.info()

https://www.google.com/search?

    q=%EC%B0%A8%EC%9D%80%EC%9A%B0&
    
    oq=%EC%B0%A8%EC%9D%80%EC%9A%B0&
    
    aqs=chrome..69i57j0l5.1862j0j8&
    
    sourceid=chrome&
    
    ie=UTF-8

https://www.google.com/search?q=%EC%B0%A8%EC%9D%80%EC%9A%B0&

In [25]:
resp = request.urlopen("https://www.google.com/search?q=%EC%B0%A8%EC%9D%80%EC%9A%B0&")
resp.code, resp.reason                        

HTTPError: HTTP Error 403: Forbidden

-> 정상적인 url이 아니라 봇이라고 판단하고 차단된다.

에러 핸들링

In [26]:
from urllib import error

try:
    resp = request.urlopen("https://www.google.com/search?q=%EC%B0%A8%EC%9D%80%EC%9A%B0&")
except error.HTTPError as e:
    print(e.code, e.reason, e.headers)

403 Forbidden Content-Type: text/html; charset=UTF-8
Date: Thu, 11 Jul 2019 02:33:34 GMT
Server: gws
Cache-Control: private
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Alt-Svc: quic=":443"; ma=2592000; v="46,43,39"
Accept-Ranges: none
Vary: Accept-Encoding
Connection: close




봇인걸 숨겨보자

내 User-Agent를 알아보자

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36

In [33]:
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

In [31]:
request.urlopen??

-> 핸들링 할 수 있는게 없다.

In [30]:
request.Request??

In [35]:
try:
    req = request.Request("https://www.google.com/search?q=%EC%B0%A8%EC%9D%80%EC%9A%B0&",
                          headers=headers)
    resp = request.urlopen(req)
except error.HTTPError as e:
    print(e.code, e.reason, e.headers)

In [40]:
resp.code, resp.reason, req.headers

(200,
 'OK',
 {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'})